# Evaluate models on test set and compare metrics

In [ ]:
import os
import numpy as np
from scipy.special import softmax
from sklearn.decomposition import PCA

import src.CLVAE as CLVAE

# Define Model and Hyperparameters

In [2]:
config_options = {
    "mnist": {
        'dataset': 'mnist',
        "network": {
            "latent_dim": 2,
            "intermediate_dims": [[32,3,2],[64,3,2]]
        },
    },
    "fashion_mnist": {
        'dataset': 'fashion_mnist',
        "network": {
            "latent_dim": 3,
            "intermediate_dims": [[16,3,1],[32,3,2],[64,3,2]]
        },
    },
    "cifar10": {
        'dataset': 'cifar10',
        "network": {
            "latent_dim": 3,
            "intermediate_dims": [[64,3,2],[128,3,2],[256,3,2]]
        },
    }
}

In [6]:
config = config_options['mnist']

print("Loading Data")
x_train, y_train, x_test, y_test, x_anom, y_anom, config = CLVAE.load_data(config)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(x_anom.shape, y_anom.shape)

anomalous_digit = config['anomalous_digit']
dataset = config['dataset']
if config['network']['latent_dim'] == 3: dataset += "_dim3"
os.makedirs("confidences/" + dataset, exist_ok=True)

Loading Data
(54077, 28, 28, 1) (54077,)
(9020, 28, 28, 1) (9020,)
(6903, 28, 28, 1) (6903,)


# Calc confidences CLVAE

In [ ]:
path = "models/" + dataset + "/clvae"
files = sorted(os.listdir(path))
latent_div_conf = np.zeros([y_anom.shape[0]*2, 10, len(files)])
mse_conf = np.zeros([y_anom.shape[0]*2, len(files)])

model, encoder, decoder = CLVAE.build_clvae(config)

idx = 0
for f in files:
    print(f)
    model.load_weights(path + "/" + f + "/model")
    encoder.load_weights(path + "/" + f + "/encoder")
    decoder.load_weights(path + "/" + f + "/decoder")

    print("Predicting Result")
    z_train_clvae = encoder.predict(x_train)
    z_test_clvae = encoder.predict(x_test)
    z_anom_clvae = encoder.predict(x_anom)

    x_rec_test_clvae = decoder.predict(z_test_clvae)
    x_rec_anom_clvae = decoder.predict(z_anom_clvae)


    def get_conf(config, z_train, y_train, z_test, y_test, z_anom, y_anom):
        mu, sigma, labels, max_pdf = CLVAE.calc_distribution(config, z_train, y_train)

        y_full_test = np.zeros(y_test.shape)
        y_full_test[y_test < anomalous_digit] = y_test[y_test < anomalous_digit]
        y_full_test[y_test >= anomalous_digit] = y_test[y_test >= anomalous_digit] + 1
        N_norm = y_anom.shape[0]
        y_full_test = np.hstack([y_full_test[:N_norm], y_anom])
        z_full_test = np.vstack([z_test[:N_norm, :], z_anom])


        conf = np.zeros([z_full_test.shape[0], 10])
        for j in range(z_full_test.shape[0]):
            for i in range(9):
                cls = i
                if i >= anomalous_digit: cls += 1
                x = z_full_test[j,:] - mu[i,:]
                sigma_inv = np.linalg.inv(sigma[i])
                conf[j, cls] = np.matmul(np.matmul(x, sigma_inv), x)

        conf = CLVAE.get_confidences(config, z_full_test, mu, sigma, max_pdf)
        
        return conf
    
    def get_mse(x_test, rec_test, y_test, x_anom, rec_anom, y_anom):

        y_full_test = np.zeros(y_test.shape)
        y_full_test[y_test < anomalous_digit] = y_test[y_test < anomalous_digit]
        y_full_test[y_test >= anomalous_digit] = y_test[y_test >= anomalous_digit] + 1
        N_norm = y_anom.shape[0]
        y_full_test = np.hstack([y_full_test[:N_norm], y_anom])

        full_size = config['network']['original_dim']*config['network']['color_dim']
        err = x_test - rec_test
        err = np.reshape(err, [-1, full_size])
        mse_test = np.mean(err, axis=1)
        err = x_anom - rec_anom
        err = np.reshape(err, [-1, full_size])
        mse_anom = np.mean(err, axis=1)
        mse = np.hstack([mse_test[:N_norm], mse_anom])
        return mse

    latent_div_conf[:, :, idx] = get_conf(config, z_train_clvae, y_train, z_test_clvae, y_test, z_anom_clvae, y_anom)
    mse_conf[:, idx] = get_mse(x_test, x_rec_test_clvae, y_test, x_anom, x_rec_anom_clvae, y_anom)
    idx += 1


In [8]:
np.save("confidences/" + dataset + "/clvae_ld", latent_div_conf)
np.save("confidences/" + dataset + "/clvae_mse", mse_conf)

# Confidences VAE

In [ ]:

path = "models/" + dataset + "/vae"
files = sorted(os.listdir(path))
mse_conf = np.zeros([y_anom.shape[0]*2, len(files)])


model, encoder, decoder = CLVAE.build_vae(config)

full_size = config['network']['original_dim']*config['network']['color_dim']

idx = 0
for f in files:
    print(f)
    model.load_weights(path + "/" + f + "/model")
    encoder.load_weights(path + "/" + f + "/encoder")
    decoder.load_weights(path + "/" + f + "/decoder")

    print("Predicting Result")
    z_test_clvae = encoder.predict(x_test)
    z_anom_clvae = encoder.predict(x_anom)

    if len(z_test_clvae) == 3: z_test_clvae = z_test_clvae[2]
    if len(z_anom_clvae) == 3: z_anom_clvae = z_anom_clvae[2]

    x_rec_test_clvae = decoder.predict(z_test_clvae)
    x_rec_anom_clvae = decoder.predict(z_anom_clvae)
    
    def get_mse(x_test, rec_test, y_test, x_anom, rec_anom, y_anom):

        N_norm = y_anom.shape[0]

        err = x_test - rec_test
        err = np.reshape(err, [-1, full_size])
        mse_test = np.mean(err**2, axis=1)

        err = x_anom - rec_anom
        err = np.reshape(err, [-1, full_size])
        mse_anom = np.mean(err**2, axis=1)
        
        mse = np.hstack([mse_test[:N_norm], mse_anom])
        return mse

    mse_conf[:, idx] = get_mse(x_test, x_rec_test_clvae, y_test, x_anom, x_rec_anom_clvae, y_anom)
    #print(get_auc(mse_conf[:, idx], 0, np.max(mse_conf[:, idx])))
    idx += 1


In [6]:
np.save("confidences/" + dataset + "/vae_mse", mse_conf)

# Confidence CNN

In [ ]:

path = "models/" + dataset + "/cnn"
files = sorted(os.listdir(path))
scores = np.zeros([y_anom.shape[0]*2, len(files)])


model = CLVAE.build_cnn(config)
model.summary()

idx = 0
for f in files:
    print(f)
    model.load_weights(path + "/" + f + "/model")

    print("Predicting Result")
    conf_test = model.predict(x_test)
    conf_anom = model.predict(x_anom)
    
    N_norm = y_anom.shape[0]
    score_test = -np.max(softmax(conf_test, axis=1), axis=1)
    score_anom = -np.max(softmax(conf_anom, axis=1), axis=1)
    score = np.hstack([score_test[:N_norm], score_anom])

    scores[:, idx] = score
    #print(get_auc(mse_conf[:, idx], 0, np.max(mse_conf[:, idx])))
    idx += 1


In [4]:
np.save("confidences/" + dataset + "/cnn_score", scores)

# Confidence PCA

In [28]:
# PCA
y_full_test = np.zeros(y_test.shape)
y_full_test[y_test < anomalous_digit] = y_test[y_test < anomalous_digit]
y_full_test[y_test >= anomalous_digit] = y_test[y_test >= anomalous_digit] + 1
N_norm = y_anom.shape[0]
y_full_test = np.hstack([y_full_test[:N_norm], y_anom])
x_full_test = np.vstack([x_test[:N_norm, :], x_anom])
print(x_full_test.shape, y_full_test.shape)
Xtrain = np.reshape(x_train, (-1, config['network']['original_dim']*config['network']['color_dim']))
Xtest = np.reshape(x_full_test, (-1, config['network']['original_dim']*config['network']['color_dim']))
print(Xtrain.shape)

scores = np.zeros((Xtest.shape[0], 9))
for i in range(9):
    pca = PCA(n_components=config['network']['latent_dim'])
    pca.fit(Xtrain[y_train == i])
    scores[:, i] = pca.score_samples(Xtest)
max_score = np.max(scores, axis=1)
score = -max_score

(13806, 28, 28, 1) (13806,)
(54077, 784)


In [29]:
np.save("confidences/" + dataset + "/pca_score", score)